In [1]:
import collections
import copy

def l():
    return [int(x) for x in open("15.txt").read().split(',')]


class m:
    idgen=0
    
    #get op and modes of following 3 addresses
    def ix(s,x):
        assert(x>=0)
        x="%05d"%(x)
        op=int(x[3:])
        m3,m2,m1=[int(c) for c in x[:3]]
        return op,m1,m2,m3


    def gv(s,p,ip,m,ix):
        assert(ix in [1,2,3])
        if m==0:#reg
            return p[p[ip+ix]]
        elif m==1:#imm
            return p[ip+ix] 
        elif m==2:#reg+rb
            return p[p[ip+ix]+s.rb]
        assert(0)
        
            
        

    
    def __init__(s,p):
        #load to mem
        s.p=collections.defaultdict(int)
        for i,v in enumerate(p):
            s.p[i]=v
            
        #possible data source
        s.supplier=None
        
        #input queue
        s.i=[]
        
        #id
        s.id=m.idgen
        m.idgen+=1
        
        #iterator for execution
        #s.it=s.iterator()
        s.ip=0
        
        #print("M",s.id,"created")
        
        #relative base
        s.rb=0
        
    
    def sendInput(s, i):
        s.i.append(i)
        
    
    def getData(s):
        ip=s.ip
        
        while 1:
            op,m1,m2,m3=s.ix(s.p[ip])
            
            if op==1:#add
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)+s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==2:#mul
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)*s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==3:#in
                assert(m1 in [0,2])
                r=None
                if len(s.i)>0:
                    r=s.i[0]
                    s.i=s.i[1:]
                else:
                    if s.supplier:
                        r=s.supplier.getData()
                    else:
                        print("id",s.id, "returning zero")
                        #assert(0)
                        r=0
                tix=s.p[ip+1]
                if m1==2:
                    tix+=s.rb
                s.p[tix]=r
                ip+=2
            elif op==4:#out
                
                r=s.gv(s.p,ip,m1,1)
                #print(s.id,"spits out",r)
                s.ip=ip+2
                return r
                #print("--id:",s.id,"onwards")
                
                
            elif op==5:#jump-if-true
                if s.gv(s.p,ip,m1,1):
                    ip=s.gv(s.p,ip,m2,2)
                else:
                    ip+=3
            elif op==6:#jump-if-false
                if not s.gv(s.p,ip,m1,1):
                    ip=s.gv(s.p,ip,m2,2)
                else:
                    ip+=3
            elif op==7:#less-than
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)<s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==8:#equals
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)==s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==9:#adjust relative base
                s.rb+=s.gv(s.p,ip,m1,1)
                ip+=2
            elif op in [99]:#end
                #print("fin", s.id)
                break
            else:
                print("id", s.id, "op",op,"c",c)
                assert(0)
            #print(op)
            
        
        return -1

In [2]:




def p1():
    M={}
    
    e=[]
    e.append((0,0,0,m(l())))
    
    while len(e):
        x,y,d,mm=e[0]
        e=e[1:]
        #print(x,y,d)
        M[(x,y)]=1
        #nswe
        for dx,dy,v in [(0,-1,1),(0,1,2),(-1,0,3),(1,0,4)]:
            mm2=copy.deepcopy(mm)
            mm2.sendInput(v)
            o=mm2.getData()
            if o==0: #wall
                #print("wall")
                pass
            elif o==1: #free
                #print("free")
                xx=x+dx
                yy=y+dy
                if not (xx,yy) in M:
                    e.append((xx,yy,d+1,mm2))
            elif o==2:
                #print("oxygen")
                return d+1 #oxygen
            else:
                print("o=",o)
                assert(0)
            
            
        
        
    return -1
    
    
    
    
   



print("part1:", p1(),298)








part1: 298 298


In [3]:
def p2():
    
    M={}
    
    e=[]
    e.append((0,0,0,m(l())))
    
    ox,oy=0,0
    
    while len(e):
        x,y,d,mm=e[0]
        e=e[1:]
        #print(x,y,d)
        M[(x,y)]=-1
        #nswe
        for dx,dy,v in [(0,-1,1),(0,1,2),(-1,0,3),(1,0,4)]:
            mm2=copy.deepcopy(mm)
            mm2.sendInput(v)
            o=mm2.getData()
            if o==0: #wall
                #print("wall")
                pass
            elif o>0: #free
                #print("free")
                xx=x+dx
                yy=y+dy
                if not (xx,yy) in M:
                    e.append((xx,yy,d+1,mm2))
                if o==2:
                    #print("oxygen")
                    ox,oy=xx,yy
    
    #regular dict instead of defaultdict
    MM={}
    for k in M:
        MM[k]=M[k]
    M=MM
    
    #print(ox,oy)
    s=[(ox,oy,0)]
    d=0
    while len(s):
        x,y,d=s[0]
        s=s[1:]
        for dx,dy in [(1,0),(-1,0),(0,1),(0,-1)]:
            xx=x+dx
            yy=y+dy
            p=(xx,yy)
            if p in M and M[p] == -1:
                del M[p]
                s.append((xx,yy,d+1))
    return d           
            
    
            
        
        
    

print("part2:", p2(),346)


part2: 346 346
